In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from alphaviz.contrib.ms_viz import MS_Viz

from peptdeep.pretrained_models import ModelManager

model_mgr = ModelManager(mask_modloss=True)
model_mgr.load_installed_models("phos")
ms_viz = MS_Viz(model_mgr)

In [3]:
# path to the .raw folder or .hdf file
raw_name = r"yanliu_20180406_Phos_Noco_5"
raw_file = r'd:\AlphaDeepData\DeepPhospho\Phos_PXD017476\raw\yanliu_20180406_Phos_Noco_5.raw'
hdf = raw_file+".hdf"
# path to the DIA-NN output folder
diann_tsv = r'D:\AlphaDeepData\DeepPhospho\Phos_PXD017476\diann\diann-report.tsv'
model_mgr.instrument = 'QE'
model_mgr.nce = 30

In [4]:
import os

if os.path.isfile(hdf):
    ms_viz.load_ms_data(hdf, "alpharaw_hdf", dda=False)
else:
    ms_viz.load_ms_data(raw_file, 'thermo_raw', dda=False)
    ms_viz.tims_match.raw_data.save_hdf(hdf)


In [5]:
tims = ms_viz.tims_data

In [6]:
ms_viz.load_psms(
    diann_tsv, 'diann', 
)
ms_viz.psm_df

,raw_name,sequence,charge,rt,rt_start,rt_stop,mobility,proteins,uniprot_ids,genes,scan_num,score,fdr,spec_idx,mods,mod_sites,nAA,rt_norm,precursor_mz,ccs
0,yanliu_20180406_Phos_Noco_5,MSFSSLK,2,66.0922,65.4704,66.3659,0,MERL_HUMAN,P35240,NF2,47365,0.995291,0.000364,47364,Phospho@S,2,7,0.492663,440.187730,0.0
1,yanliu_20180406_Phos_Noco_5,VNLSPNR,2,27.1092,26.6903,27.5131,0,IRS1_HUMAN,P35568,IRS1,19525,0.998135,0.000221,19524,Phospho@S,4,7,0.202077,440.207842,0.0
2,yanliu_20180406_Phos_Noco_5,TKPIWTR,2,43.1453,42.4249,43.4874,0,H90B2_HUMAN;HS902_HUMAN;HS90A_HUMAN;HS90B_HUMAN,Q14568;Q58FG0;P08238;Q58FF8;P07900,HSP90AA1;HSP90AA2P;HSP90AB1;HSP90AB2P,31169,0.962936,0.002406,31168,Phospho@T,1,7,0.321613,491.249510,0.0
3,yanliu_20180406_Phos_Noco_5,TCFSPNR,2,28.3949,27.9375,28.8160,0,EIF3F_HUMAN,O00303,EIF3F,20408,0.970795,0.002019,20407,Carbamidomethyl@C;Phospho@S,2;4,7,0.211661,481.183510,0.0
4,yanliu_20180406_Phos_Noco_5,NSPLVSR,2,22.1418,21.9145,22.3693,0,MDHM_HUMAN,P40926,MDH2,16124,0.998121,0.000221,16123,Phospho@S,2,7,0.165049,426.702393,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34528,yanliu_20180406_Phos_Noco_5,LDIPDLMNEGSPVPIETGNVNIVGISYQPR,3,125.5330,125.2510,125.9490,0,CRBG3_HUMAN,Q68DQ2,CRYBG3,89998,0.999244,0.000196,89997,Phospho@S,11,30,0.935745,1106.542296,0.0
34529,yanliu_20180406_Phos_Noco_5,AGEPNSPDAEEANSPDVTAGCDPAGVHPPR,2,57.6047,57.1606,58.3218,0,NSUN2_HUMAN,Q08J23,NSUN2,41279,0.997631,0.000263,41278,Phospho@T;Carbamidomethyl@C,18;21,30,0.429396,1547.645845,0.0
34530,yanliu_20180406_Phos_Noco_5,AGEPNSPDAEEANSPDVTAGCDPAGVHPPR,2,58.8203,58.1026,59.5394,0,NSUN2_HUMAN,Q08J23,NSUN2,42159,0.990596,0.000612,42158,Carbamidomethyl@C,21,30,0.438457,1507.662679,0.0
34531,yanliu_20180406_Phos_Noco_5,HDDSSDNFCEADDIQSPEAEYVDLLLNPER,3,119.4340,118.9650,119.6590,0,ERO1A_HUMAN,Q96HE7,ERO1A,85799,0.999642,0.000151,85798,Carbamidomethyl@C,9,30,0.890282,1165.170594,0.0


# Plot

In [7]:
ith_psm = 29199
psm = ms_viz.psm_df.loc[ith_psm:ith_psm,:]
psm[[
    "sequence","mods","mod_sites","charge","rt","precursor_mz"
]]

,sequence,mods,mod_sites,charge,rt,precursor_mz
29199,GDPPRLSPDPVAGSAVSQELR,Phospho@S,7,3,74.5151,743.361752


In [8]:
model_mgr.verbose = False
ms_viz.remove_unlabeled_fragments = False
pep_frag_df = ms_viz.predict_one_pep_frag_df(psm)
ms2_fig = ms_viz.plot_mirror_ms2(
    pep_frag_df,
    plot_unmatched_peaks=True,
)
ms2_fig.show()

In [9]:
ms_viz.xic_1d_plot.rt_sec_win = 120
ms_viz.plot_elution_profile(
    pep_frag_df,
    include_ms1=False
)